The idea is that you should get the same results as when running gpbp_showcase

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from pisa.administrative_area import AdministrativeArea
from pisa.facilities import Facilities 
from pisa.population import Population
from pisa.isopolygons import OsmIsopolygonCalculator

from pisa.rename_this_module import population_served

from optimization import jg_opt
from functools import partial

import osmnx as ox

## Defining the Administrative Area

In [ ]:
admin_area = AdministrativeArea(
    country_name="Timor-Leste", admin_level=1, admin_area_name="Baucau")


In [ ]:
# retrieve the bounday (Polygon or Multipolygon) of the administrative area. 
# This is needed to build Facilties and Population

admin_area_boundaries = admin_area.get_boundaries()

In [ ]:
iso3_country_code = admin_area.get_iso3_country_code()

## Retrieve Facility data

In [ ]:
# create instance of facilities class with same tags as the gpbp_showcase notebook, not default {building: hospital}

facilities = Facilities(admin_area_boundaries, tags={"amenity":"hospital"})

existing_facilities_gdf = facilities.get_existing_facilities()

potential_facilities_gdf = facilities.estimate_potential_facilities()

In [2]:
# TODO: add plot

In [5]:
# TODO: check equality with gpbp_showcase (except maybe for indices)

## Retrieve Population Data

Using world_pop as in gpbp_showcase notebook

In [ ]:

population_gdf = Population(
    data_source="world_pop",
    iso3_country_code=iso3_country_code,
    admin_area_boundaries=admin_area_boundaries,
).get_population_gdf()

In [ ]:
# TODO: add plot

In [ ]:
# TODO: check equality with gpbp_showcase (except maybe for indices)

## Compute isopolygons using OSM
because that is what's used in the notebook.

TODO: 

- check Mapbox also works
- check other distance_types works
- check other mode_of_transport works

In [18]:
# some constants 

DISTANCE_TYPE = "length" # using this notation as it's used in gpbp_showcase
distance_values = [2000, 5000, 10000]

mode_of_transport = "driving" 



Since strategy is OSM, you need a road_network.

Since mode_of_transport isn't time, you don't need to add_time_to_edges.

In [ ]:
# I logically don't know where this belongs ATM

road_network = ox.graph_from_polygon(polygon=admin_area_boundaries, network_type=mode_of_transport)

# I also don't know where to place add_time_to_edges

In [ ]:
isopolygons_existing_facilities = OsmIsopolygonCalculator(
    facilities_lon_lat = existing_facilities_gdf[["longitude", "latitude"]], # double check
    distance_type=DISTANCE_TYPE, 
    distance_values=distance_values,
    road_network=road_network
).calculate_isopolygons()



In [ ]:
isopolygons_potential_facilities = OsmIsopolygonCalculator(
    facilities_lon_lat = potential_facilities_gdf[["longitude", "latitude"]], 
    distance_type=DISTANCE_TYPE, 
    distance_values=distance_values,
    road_network=road_network
).calculate_isopolygons()


In [ ]:
# TODO: check equality with gpbp_showcase

## Prepare optimization data

Preparing the variables that go into the cell below

In [ ]:
# I think you just need to convert the population column of population_gdf into an np.array. 
# *TODO*: DOUBLE CHECK


population_count = population_gdf.population.values

In [ ]:
current = population_served(
    population_gdf=population_gdf,
    facilites_gdf=existing_facilities_gdf, # do you need this param?
    isopolygons=isopolygons_existing_facilities,
)

In [ ]:
potential = population_served(
    population_gdf=population_gdf,
    facilites_gdf=potential_facilities_gdf, # do you need this param?
    isopolygons=isopolygons_potential_facilities
)

In [ ]:
CBC_SOLVER_PATH = None # fill out the solver path where the cbc executable 
BUDGET = [5, 20, 50] # budget for the optimization in terms of how many locations can be built
# DISTANCE_TYPE = "length" # defined in a cell above

cbc_optimize = partial(
                    jg_opt.OpenOptimize, solver_path=CBC_SOLVER_PATH
                )
jg_opt.Solve(household=population_count, 
             current=current, 
             potential=potential, 
             accessibility=DISTANCE_TYPE, 
             budgets=BUDGET, 
             optimize=cbc_optimize, 
             type='ID')

(       10000      5000      2000
 5   0.387986  0.256426  0.126729
 20  0.392613  0.266551  0.134299
 50  0.392613  0.268571  0.135708,
                                                 10000  \
 5                                [45, 47, 63, 30, 37]   
 20  [45, 47, 63, 30, 37, 38, 42, 55, 56, 34, 58, 2...   
 50  [45, 47, 63, 30, 37, 38, 42, 55, 56, 34, 58, 2...   
 
                                                  5000  \
 5                                [47, 42, 45, 29, 63]   
 20  [47, 42, 45, 29, 63, 36, 38, 62, 70, 58, 59, 4...   
 50  [47, 42, 45, 29, 63, 36, 38, 62, 70, 58, 59, 4...   
 
                                                  2000  
 5                                [59, 63, 45, 27, 42]  
 20  [59, 63, 45, 27, 42, 47, 44, 54, 46, 33, 52, 3...  
 50  [59, 63, 45, 27, 42, 47, 44, 54, 46, 33, 52, 3...  )